In [50]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

import numpy as np

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn


In [3]:
db, conn = pgconnect(credentials)

Connected successfully.


In [33]:
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result


In [34]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.0 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [51]:
SA2_data = gpd.read_file("SA2_2016_AUST/SA2_2016_AUST.shp")
SA2_data = SA2_data.replace(to_replace='None', value=np.nan).dropna()
SA2_data

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,"POLYGON ((149.58423 -35.44427, 149.58444 -35.4..."
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,"POLYGON ((149.21898 -35.36739, 149.21799 -35.3..."
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634,"POLYGON ((149.21325 -35.34325, 149.21619 -35.3..."
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034,"POLYGON ((149.24033 -35.34782, 149.24023 -35.3..."
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099,"POLYGON ((149.23580 -35.38738, 149.23771 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,"POLYGON ((148.80406 -35.37620, 148.80416 -35.3..."
2304,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,"POLYGON ((105.67392 -10.41567, 105.67398 -10.4..."
2305,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,"MULTIPOLYGON (((96.83047 -12.17636, 96.83045 -..."
2306,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,"MULTIPOLYGON (((150.69566 -35.18297, 150.69555..."


In [52]:
srid = 4283
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)


In [44]:
SA2_data_og = SA2_data.copy()  
SA2_data['geom'] = SA2_data['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  
SA2_data = SA2_data.drop(columns="geometry") 
SA2_data

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geom
0,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3418.3525,MULTIPOLYGON (((149.58423299700007 -35.4442699...
1,101021008,11008,Karabar,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,6.9825,MULTIPOLYGON (((149.2189819130001 -35.36739402...
2,101021009,11009,Queanbeyan,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,4.7634,MULTIPOLYGON (((149.2132479820001 -35.34324799...
3,101021010,11010,Queanbeyan - East,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,13.0034,MULTIPOLYGON (((149.240332114 -35.347822616999...
4,101021011,11011,Queanbeyan Region,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,3054.4099,MULTIPOLYGON (((149.23580096700005 -35.3873753...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,801111141,81141,Namadgi,80111,Urriarra - Namadgi,801,Australian Capital Territory,8ACTE,Australian Capital Territory,8,Australian Capital Territory,1202.8930,MULTIPOLYGON (((148.80406400200002 -35.3762040...
2304,901011001,91001,Christmas Island,90101,Christmas Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,136.1356,MULTIPOLYGON (((105.67391901000008 -10.4156730...
2305,901021002,91002,Cocos (Keeling) Islands,90102,Cocos (Keeling) Islands,901,Other Territories,9OTER,Other Territories,9,Other Territories,13.7163,MULTIPOLYGON (((96.83046599400006 -12.17635995...
2306,901031003,91003,Jervis Bay,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,67.8134,MULTIPOLYGON (((150.6956629970001 -35.18296700...


In [64]:
conn.execute("""
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2 (
    "SA2_MAIN16" NUMERIC, 
    "SA2_5DIG16" NUMERIC, 
    "SA2_NAME16" VARCHAR(50), 
    "SA3_CODE16" NUMERIC, 
    "SA3_NAME16" VARCHAR(50),
    "SA4_CODE16" NUMERIC,
    "SA4_NAME16" VARCHAR(50),
    "GCC_CODE16" VARCHAR(10),
    "GCC_NAME16" VARCHAR(50),
    "STE_CODE16" NUMERIC,
    "STE_NAME16" VARCHAR(50),
    "AREASQKM16" NUMERIC,
    geom GEOMETRY(MULTIPOLYGON,4283)
);"""
)

In [66]:
SA2_data.to_sql("sa2", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from sa2")

,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geom
0,101021007.0,11007.0,Braidwood,10102.0,Queanbeyan,101.0,Capital Region,1RNSW,Rest of NSW,1.0,New South Wales,3418.3525,0106000020BB100000010000000103000000010000005F...
1,101021008.0,11008.0,Karabar,10102.0,Queanbeyan,101.0,Capital Region,1RNSW,Rest of NSW,1.0,New South Wales,6.9825,0106000020BB100000010000000103000000010000002C...
2,101031015.0,11015.0,Cooma Region,10103.0,Snowy Mountains,101.0,Capital Region,1RNSW,Rest of NSW,1.0,New South Wales,6250.8748,0106000020BB100000010000000103000000020000006E...
3,101031016.0,11016.0,Jindabyne - Berridale,10103.0,Snowy Mountains,101.0,Capital Region,1RNSW,Rest of NSW,1.0,New South Wales,3939.5484,0106000020BB100000010000000103000000010000005F...
4,101041017.0,11017.0,Batemans Bay,10104.0,South Coast,101.0,Capital Region,1RNSW,Rest of NSW,1.0,New South Wales,63.7074,0106000020BB1000000100000001030000000100000036...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4579,801111141.0,81141.0,Namadgi,80111.0,Urriarra - Namadgi,801.0,Australian Capital Territory,8ACTE,Australian Capital Territory,8.0,Australian Capital Territory,1202.8930,0106000020BB100000010000000103000000010000000C...
4580,901011001.0,91001.0,Christmas Island,90101.0,Christmas Island,901.0,Other Territories,9OTER,Other Territories,9.0,Other Territories,136.1356,0106000020BB1000000100000001030000000100000094...
4581,901021002.0,91002.0,Cocos (Keeling) Islands,90102.0,Cocos (Keeling) Islands,901.0,Other Territories,9OTER,Other Territories,9.0,Other Territories,13.7163,0106000020BB100000380000000103000000010000000E...
4582,901031003.0,91003.0,Jervis Bay,90103.0,Jervis Bay,901.0,Other Territories,9OTER,Other Territories,9.0,Other Territories,67.8134,0106000020BB1000000300000001030000000100000017...


In [45]:
schoolsdata = pd.concat(
    map(gpd.read_file, ['school_catchments/catchments_primary.shp', 'school_catchments/catchments_secondary.shp', 'school_catchments/catchments_future.shp']), ignore_index=True)
schoolsdata

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.93564 -33.71612, 150.93715 -33.7..."
1,3943,PRIMARY,Gordon WPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.14085 -33.74463, 151.14113 -33.7..."
2,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.18336 -33.74748, 151.18443 -33.7..."
3,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.04518 -33.77303, 151.04526 -33.7..."
4,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.92567 -33.72960, 150.92602 -33.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,4680,PRIMARY,Googong PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,NaN,"POLYGON ((149.26653 -35.49838, 149.26703 -35.5..."
2141,2540,PRIMARY,Michelago PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,NaN,"POLYGON ((149.14237 -35.56872, 149.16075 -35.5..."
2142,8922,HIGH_COED,Jerrabomberra HS,20220305,0,0,0,0,0,0,0,2023,2023,0,0,0,0,NaN,"POLYGON ((149.19808 -35.38147, 149.19852 -35.3..."
2143,8919,HIGH_COED,Bungendore HS,20220305,0,0,0,0,0,0,0,2023,2023,0,0,0,0,NaN,"POLYGON ((149.24085 -34.91439, 149.24367 -34.9..."


In [46]:
schoolsdata_og = schoolsdata.copy()  
schoolsdata['geom'] = schoolsdata['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  
schoolsdata = schoolsdata.drop(columns="geometry") 
schoolsdata

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geom
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((150.93563850416004 -33.7161211...
1,3943,PRIMARY,Gordon WPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.14085043012102 -33.7446267...
2,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.1833640465581 -33.74748398...
3,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((151.0451821055135 -33.77303212...
4,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,MULTIPOLYGON (((150.92567327976582 -33.7296030...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,4680,PRIMARY,Googong PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,NaN,MULTIPOLYGON (((149.26653458527704 -35.4983768...
2141,2540,PRIMARY,Michelago PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,NaN,MULTIPOLYGON (((149.14237305771536 -35.5687160...
2142,8922,HIGH_COED,Jerrabomberra HS,20220305,0,0,0,0,0,0,0,2023,2023,0,0,0,0,NaN,MULTIPOLYGON (((149.1980792584528 -35.38147320...
2143,8919,HIGH_COED,Bungendore HS,20220305,0,0,0,0,0,0,0,2023,2023,0,0,0,0,NaN,MULTIPOLYGON (((149.2408471104619 -34.91439387...


In [53]:
conn.execute("""
DROP TABLE IF EXISTS schools;
CREATE TABLE schools (
    "USE_D" NUMERIC, 
    "CATCH_TYPE" VARCHAR(10), 
    "USE_DESC" VARCHAR(20), 
    "ADD_DATE" NUMERIC, 
    "KINDERGART" VARCHAR(10),
    "YEAR1" VARCHAR(10),
    "YEAR2" VARCHAR(10),
    "YEAR3" VARCHAR(10),
    "YEAR4" VARCHAR(10),
    "YEAR5" VARCHAR(10),
    "YEAR6" VARCHAR(10),
    "YEAR7" VARCHAR(10),
    "YEAR8" VARCHAR(10),
    "YEAR9" VARCHAR(10),
    "YEAR10" VARCHAR(10),
    "YEAR11" VARCHAR(10),
    "YEAR12" VARCHAR(10),
    "PRIORITY" VARCHAR(10),
    geom GEOMETRY(MULTIPOLYGON,4283)
);"""
)

In [54]:
schoolsdata.to_sql("schools", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from schools")

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geom
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB1000000100000001030000000100000078...
1,3943,PRIMARY,Gordon WPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB100000010000000103000000010000006F...
2,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB10000001000000010300000001000000BE...
3,1839,PRIMARY,Empire Bay PS,20211115,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB1000000100000001030000000100000032...
4,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020BB1000000100000001030000000100000065...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,8524,HIGH_COED,Karabar HS,20220305,0,0,0,0,0,0,0,2023,2023,2023,2023,2023,2023,None,0106000020BB100000010000000103000000010000006E...
2141,4680,PRIMARY,Googong PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,None,0106000020BB1000000100000001030000000100000072...
2142,2540,PRIMARY,Michelago PS,20220305,2023,2023,2023,2023,2023,2023,2023,0,0,0,0,0,0,None,0106000020BB10000001000000010300000001000000BF...
2143,8922,HIGH_COED,Jerrabomberra HS,20220305,0,0,0,0,0,0,0,2023,2023,0,0,0,0,None,0106000020BB1000000100000001030000000100000020...


In [55]:
breakdata = gpd.read_file("break_and_enter/BreakEnterDwelling_JanToDec2021.shp")
breakdata = breakdata.replace(to_replace='None', value=np.nan).dropna()
breakdata

,OBJECTID,Contour,Density,ORIG_FID,Shape_Leng,Shape_Area,geometry
0,1,8.0,Low Density,1,0.012138,0.000006,"POLYGON ((149.91078 -37.06636, 149.91080 -37.0..."
1,2,8.0,Low Density,1,0.019106,0.000015,"POLYGON ((149.90601 -37.05837, 149.90602 -37.0..."
2,3,8.0,Low Density,1,0.006068,0.000002,"POLYGON ((148.94250 -37.04209, 148.94253 -37.0..."
3,4,8.0,Low Density,1,0.011006,0.000006,"POLYGON ((149.19583 -36.91833, 149.19584 -36.9..."
4,5,8.0,Low Density,1,0.008073,0.000003,"POLYGON ((149.84836 -36.68459, 149.84838 -36.6..."
...,...,...,...,...,...,...,...
2589,2590,10.0,High Density,3,0.015358,0.000011,"POLYGON ((153.27961 -28.55388, 153.27964 -28.5..."
2590,2591,10.0,High Density,3,0.028327,0.000029,"POLYGON ((150.35939 -28.60530, 150.35941 -28.6..."
2591,2592,10.0,High Density,3,0.006654,0.000002,"POLYGON ((153.56825 -28.38307, 153.56828 -28.3..."
2592,2593,10.0,High Density,3,0.047098,0.000044,"POLYGON ((153.53917 -28.18858, 153.53920 -28.1..."


In [56]:
breakdata_og = breakdata.copy()  
breakdata['geom'] = breakdata['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  
breakdata = breakdata.drop(columns="geometry") 
breakdata

,OBJECTID,Contour,Density,ORIG_FID,Shape_Leng,Shape_Area,geom
0,1,8.0,Low Density,1,0.012138,0.000006,MULTIPOLYGON (((149.91078014700008 -37.0663631...
1,2,8.0,Low Density,1,0.019106,0.000015,MULTIPOLYGON (((149.9060088220001 -37.05836717...
2,3,8.0,Low Density,1,0.006068,0.000002,MULTIPOLYGON (((148.94249939000008 -37.0420853...
3,4,8.0,Low Density,1,0.011006,0.000006,MULTIPOLYGON (((149.1958326340001 -36.91833255...
4,5,8.0,Low Density,1,0.008073,0.000003,MULTIPOLYGON (((149.84836006500007 -36.6845891...
...,...,...,...,...,...,...,...
2589,2590,10.0,High Density,3,0.015358,0.000011,MULTIPOLYGON (((153.27961053500007 -28.5538784...
2590,2591,10.0,High Density,3,0.028327,0.000029,MULTIPOLYGON (((150.3593920080001 -28.60530123...
2591,2592,10.0,High Density,3,0.006654,0.000002,MULTIPOLYGON (((153.56824816100004 -28.3830681...
2592,2593,10.0,High Density,3,0.047098,0.000044,MULTIPOLYGON (((153.5391676050001 -28.18857974...


In [59]:
conn.execute("""
DROP TABLE IF EXISTS break_enter;
CREATE TABLE break_enter (
    "OBJECTID" NUMERIC, 
    "Contour" NUMERIC, 
    "Density" VARCHAR(20), 
    "ORIG_FID" NUMERIC, 
    "Shape_Leng" NUMERIC,
    "Shape_Area" NUMERIC,
    geom GEOMETRY(MULTIPOLYGON,4283)
);"""
)

In [60]:
breakdata.to_sql("break_enter", conn, if_exists='append', index=False, dtype={'geom': Geometry('MULTIPOLYGON', srid)})
query(conn, "select * from break_enter")

,OBJECTID,Contour,Density,ORIG_FID,Shape_Leng,Shape_Area,geom
0,1.0,8.0,Low Density,1.0,0.012138,0.000006,0106000020BB1000000100000001030000000100000015...
1,2.0,8.0,Low Density,1.0,0.019106,0.000015,0106000020BB100000010000000103000000010000001D...
2,3.0,8.0,Low Density,1.0,0.006068,0.000002,0106000020BB1000000100000001030000000100000007...
3,4.0,8.0,Low Density,1.0,0.011006,0.000006,0106000020BB100000010000000103000000010000000E...
4,5.0,8.0,Low Density,1.0,0.008073,0.000003,0106000020BB100000010000000103000000010000000F...
...,...,...,...,...,...,...,...
2589,2590.0,10.0,High Density,3.0,0.015358,0.000011,0106000020BB1000000100000001030000000100000016...
2590,2591.0,10.0,High Density,3.0,0.028327,0.000029,0106000020BB1000000100000001030000000100000028...
2591,2592.0,10.0,High Density,3.0,0.006654,0.000002,0106000020BB1000000100000001030000000100000009...
2592,2593.0,10.0,High Density,3.0,0.047098,0.000044,0106000020BB100000010000000103000000010000004D...


In [ ]:
conn.close()
db.dispose()